# Random Effects Estimation (Lab 1)

### Intro and objectives


### In this lab you will learn:
1. examples of random effects estimation
2. how to fit random effects models in Python


## What I hope you'll get out of this lab
* The feeling that you'll "know where to start" when you need to fit random effects models
* Worked Examples
* How to interpret the results obtained

In [20]:
!pip install wooldridge
!pip install linearmodels
import wooldridge as woo
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import linearmodels as plm
import numpy as np
import scipy.stats as stats

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Example. Does air carrier concentration increase flying prices ?

#### The data in AIRFARE are from Wooldridge (1997).

#### We are interested in determining if the concentration of air companies drive up prices for passengers.


#### Variables:

year: 1997, 1998, 1999, 2000

id: route identifier

dist: distance, in miles

passen: avg. passengers per day

fare: avg. one-way fare, $

bmktshr: fraction market, biggest carrier

ldist: log(distance)

y98: =1 if year == 1998

y99: =1 if year == 1999

y00: =1 if year == 2000

lfare: log(fare)

ldistsq: ldist^2

concen: = bmktshr

lpassen: log(passen)








#### We want to fit a model of the form:

$log(fare_{it})=\eta_t+\beta_1*concen_{i,t}+\beta_2*log(dist_{i})+\beta_3*[log(dist_{i})]^2+a_i+u_{it},t=1,\ldots,4$

#### where $\eta_t$ means that we allow for different year intercepts.

In [2]:
AirFare = woo.dataWoo('airfare')
AirFare = AirFare.set_index(['id', 'year'], drop=False)

In [3]:
AirFare.head()

year  id  dist  passen  fare  bmktshr     ldist  y98  y99  y00  \
id year                                                                   
1  1997  1997   1   528     152   106   0.8386  6.269096    0    0    0   
   1998  1998   1   528     265   106   0.8133  6.269096    1    0    0   
   1999  1999   1   528     336   113   0.8262  6.269096    0    1    0   
   2000  2000   1   528     298   123   0.8612  6.269096    0    0    1   
2  1997  1997   2   861     282   104   0.5798  6.758094    0    0    0   

            lfare    ldistsq  concen   lpassen  
id year                                         
1  1997  4.663439  39.301571  0.8386  5.023880  
   1998  4.663439  39.301571  0.8133  5.579730  
   1999  4.727388  39.301571  0.8262  5.817111  
   2000  4.812184  39.301571  0.8612  5.697093  
2  1997  4.644391  45.671837  0.5798  5.641907

In [4]:
AirFare.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4596 entries, (1, 1997) to (1149, 2000)
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   year     4596 non-null   int64  
 1   id       4596 non-null   int64  
 2   dist     4596 non-null   int64  
 3   passen   4596 non-null   int64  
 4   fare     4596 non-null   int64  
 5   bmktshr  4596 non-null   float64
 6   ldist    4596 non-null   float64
 7   y98      4596 non-null   int64  
 8   y99      4596 non-null   int64  
 9   y00      4596 non-null   int64  
 10  lfare    4596 non-null   float64
 11  ldistsq  4596 non-null   float64
 12  concen   4596 non-null   float64
 13  lpassen  4596 non-null   float64
dtypes: float64(6), int64(8)
memory usage: 557.7 KB


In [6]:
AirFare.head(10)

year  id  dist  passen  fare  bmktshr     ldist  y98  y99  y00  \
id year                                                                   
1  1997  1997   1   528     152   106   0.8386  6.269096    0    0    0   
   1998  1998   1   528     265   106   0.8133  6.269096    1    0    0   
   1999  1999   1   528     336   113   0.8262  6.269096    0    1    0   
   2000  2000   1   528     298   123   0.8612  6.269096    0    0    1   
2  1997  1997   2   861     282   104   0.5798  6.758094    0    0    0   
   1998  1998   2   861     178   105   0.5817  6.758094    1    0    0   
   1999  1999   2   861     204   115   0.7319  6.758094    0    1    0   
   2000  2000   2   861     190   129   0.5386  6.758094    0    0    1   
3  1997  1997   3   852     241   207   0.8180  6.747587    0    0    0   
   1998  1998   3   852     253   188   0.8172  6.747587    1    0    0   

            lfare    ldistsq  concen   lpassen  
id year                                         
1  1997  4.663439  39.301571  0.8386  5.023880  
   1998  4.663439  39.301571  0.8133  5.579730  
   1999  4.727388  39.301571  0.8262  5.817111  
   2000  4.812184  39.301571  0.8612  5.697093  
2  1997  4.644391  45.671837  0.5798  5.641907  
   1998  4.653960  45.671837  0.5817  5.181784  
   1999  4.744932  45.671837  0.7319  5.318120  
   2000  4.859812  45.671837  0.5386  5.247024  
3  1997  5.332719  45.529926  0.8180  5.484797  
   1998  5.236442  45.529926  0.8172  5.533390

### Let's fit a random effects model first

In [10]:
# RE model estimation:
reg_re= plm.RandomEffects.from_formula(
    formula='lfare~concen+ldist+ldistsq+y98+y99+y00', data=AirFare)

results_re = reg_re.fit()

In [11]:
results_re

Dep. Variable:,lfare,R-squared:,0.9840
Estimator:,RandomEffects,R-squared (Between):,0.9958
No. Observations:,4596,R-squared (Within):,0.1348
Date:,"Sun, Jan 15 2023",R-squared (Overall):,0.9954
Time:,08:10:39,Log-likelihood,3741.8
Cov. Estimator:,Unadjusted,,
,,F-statistic:,4.692e+04
Entities:,1149,P-value,0.0000
Avg Obs:,4.0000,Distribution:,"F(6,4590)"
Min Obs:,4.0000,,
Max Obs:,4.0000,F-statistic (robust):,4.692e+04


### Let's fit a fixed effects model

In [13]:
# FIRST FE model estimation:
reg_fe = plm.PanelOLS.from_formula(
    formula="lfare~concen+EntityEffects+TimeEffects",
    data=AirFare, drop_absorbed=True)


results_fe = reg_fe.fit()

In [14]:
results_fe

Dep. Variable:,lfare,R-squared:,0.0095
Estimator:,PanelOLS,R-squared (Between):,0.0395
No. Observations:,4596,R-squared (Within):,0.0019
Date:,"Sun, Jan 15 2023",R-squared (Overall):,0.0394
Time:,08:21:15,Log-likelihood,4435.1
Cov. Estimator:,Unadjusted,,
,,F-statistic:,32.965
Entities:,1149,P-value,0.0000
Avg Obs:,4.0000,Distribution:,"F(1,3443)"
Min Obs:,4.0000,,
Max Obs:,4.0000,F-statistic (robust):,32.965


#### For the fixed effects model only the coefficient for concentration is computed, the rest are time invariant (fixed effects models remove time invariant factors)

In [21]:
# Hausman test of FE vs. RE
# (I) find overlapping coefficients:
common_coef = set(results_fe.params.index).intersection(results_re.params.index)
     

# (II) calculate differences between FE and RE:

b_re = results_re.params
b_re_cov = results_re.cov

b_fe = results_fe.params
b_fe_cov = results_fe.cov
     


b_diff = np.array(results_fe.params[common_coef] - results_re.params[common_coef])
df = len(b_diff)
b_diff.reshape((df, 1))
b_cov_diff = np.array(b_fe_cov.loc[common_coef, common_coef] -
                      b_re_cov.loc[common_coef, common_coef])
#b_cov_diff.reshape((df, df))


# (III) calculate test statistic:
stat = abs(np.transpose(b_diff) @ np.linalg.inv(b_cov_diff) @ b_diff)
pval = 1 - stats.chi2.cdf(stat, df)


print(f'stat: {stat}\n')
print(f'pval: {pval}\n')

stat: 10.848318049097191

pval: 0.0009888553304010506



## How do we interpret the results?

#### We have fit two alternative models: fixed-effects and random-effects. 

#### 1. In both instances the coefficient for the factor concentration is large and statistically significant. 

#### 2. In the random effects model we find that 1 unit change in concentration increases air fares by 20%

#### 3. In the fixed effects model we find that 1 unit change in concentration increases air fares by 16.8%

#### 4. Based on the results of the Hausman Test (10.84, p-value=0.0009) we reject the hypotheses that both models are equivalent. We prefer the fixed effect model.